# mock-data-set-generation-to-database


- a process to follow when generating test data
- a template to generate mock data sets to CSV file(s), including the use of weighting
- a template to allow the generated data to be inserted in to a PostgreSQL database


# Example local database setup

Commands can be run in your terminal

`docker pull postgres`

`docker run --name pg-docker -e POSTGRES_PASSWORD={INSERT_CHOSEN_PASSWORD} -p 5433:5433 -d postgres`

`docker inspect pg-docker`

 - retrieve IP address from output of inspect command

`docker ps -a`

`docker exec -it {container ID from above command} bash`

`psql -h {retrieved IP address} -U postgres`

 - enter chosen password

`CREATE DATABASE test_db;`

`\c test_db`

`CREATE SCHEMA test_schema ;`

`CREATE TABLE test_table (
  id SERIAL PRIMARY KEY,
  test_username varchar NOT NULL,
  test_password varchar NOT NULL,
  test_email varchar,
  test_address varchar,
  test_gender varchar,
  test_phone_number numeric,
  test_created_date date
);`

In [ ]:
import psycopg2
import random
import pandas as pd
import csv
import getpass

from datetime import datetime, timedelta
from faker import Faker

# Helper functions

Useful section to define any useful function(s) used throughout the Notebook

In [ ]:
# example database connection for a postgres docker container

def connection(db):
    return psycopg2.connect(database=db,
                            user=getpass.getpass(prompt="Enter database user: "),
                            password=getpass.getpass(prompt="Enter database password: "),
                            host=getpass.getpass(prompt="Enter database host: "),
                            port=getpass.getpass(prompt="Enter database port: "))

In [ ]:
def get_next_value_in_sequence(list_of_sequence_numbers: list):
    for num in list_of_sequence_numbers:
        sequence_value = list_of_sequence_numbers.pop(0)
        return sequence_value

In [ ]:
def query_database_return_dataframe(db_connection: str, query: str):
    with connection(db_connection) as conn, conn.cursor() as cursor:
        print("\n Running provided query... \n")
        cursor.execute(f"""{query}""")  
        print("Query complete!\n")
        col_names = []
        for column_name in cursor.description:
            col_names.append(column_name[0])
        
        result = cursor.fetchall()
        result_df = pd.DataFrame(result, columns=col_names)
        
        return result_df

In [ ]:
def truncate_data_in_database(db_connection: str, db_table: str, cascade_required=False):
    with connection(db_connection) as conn, conn.cursor() as cursor:
        print(f"truncating - {db_table} - started\n")
        if cascade_required:
            print("CASCADE requested. . . \n")
            cursor.execute(f"""truncate {db_table} CASCADE ;""")
            conn.commit()
            print(f"truncating - {db_table} - with CASCADE completed\n")
        else:
            cursor.execute(f"""truncate {db_table} ;""")
            conn.commit()
            print(f"truncating - {db_table} - completed\n")

In [ ]:
def insert_data_to_database_from_csv(db_connection: str, table_name: str, file_name: str):
    with connection(db_connection) as conn, conn.cursor() as cursor:
        print(f"Connecting to {db_connection}\n")
        with open(file_name, 'r') as f:
            print(f"Copying data from {file_name} in to {db_connection}.{table_name}\n")
            next(f) 
            cursor.copy_from(f, table_name, sep=',')

            conn.commit()
            print("Data inserted successfully!\n")

In [ ]:
def set_sequence_in_database(db_connection: str, sequence_name: str, sequence_value: int):
    with connection(db_connection) as conn, conn.cursor() as cursor:
        print(f"Connecting to {db_connection} to set the sequence value to {sequence_value}\n")
        cursor.execute(f"""SELECT pg_catalog.setval('{sequence_name}', {sequence_value}, true);""")
        print(f"Set sequence value to {sequence_value} - now complete!\n")

In [ ]:
def view_csv_file_as_dataframe(file_name: str):
    return pd.read_csv(file_name)

# SQL commands

Useful section to define all of your SQL commands you plan to execute


In [ ]:
EXAMPLE_QUERY = """
SELECT *
FROM test_table;
"""

# Generate CSV files of mock data

Useful section to define the model(s) of data you wish to be created in a CSV file, which can then be used to populate your database or explore within the Notebook 

#### Performance:

- 1,000 records == 8ms
- 10,000 records == 4 seconds
- 100,000 records == 46 seconds
- 1,000,000 records == 12 minutes

In [ ]:
NUMBER_OF_TEST_RECORDS_TO_GENERATE = 1000

In [ ]:
fake = Faker()


def create_csv_file_of_mock_data(number_of_records: int, file_name: str):

    test_ids = [number for number in range(1, number_of_records)]

    with open(file_name, 'w', newline='') as csvfile:
        fieldnames = ['id',
                      'test_username',
                      'test_password',
                      'test_email',
                      'test_gender',
                      'test_address',
                      'test_phone_number',
                      'test_created_date']
        
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i in range(number_of_records - 1):
            
            test_genders = ['Male', 'Female']

            writer.writerow(
                {
                   
                  'id': get_next_value_in_sequence(test_ids),
                  'test_username': fake.first_name() + fake.last_name(),
                  'test_password':fake.uuid4(),
                  'test_email': fake.safe_email(),
                  'test_gender': random.choices(test_genders, weights=[50,50], k=1)[0],
                  'test_address': fake.street_address(),
                  'test_phone_number': fake.random_int(44777777777, 44777777999),
                  'test_created_date': datetime.today() - timedelta(days = 1)
                }
            )

In [ ]:
create_csv_file_of_mock_data(NUMBER_OF_TEST_RECORDS_TO_GENERATE, 'test_mock_data_file.csv')

In [ ]:
view_csv_file_as_dataframe('test_mock_data_file.csv')

# Insert generated data from CSV file

In [ ]:
insert_data_to_database_from_csv('test_db',
                                 'test_table',
                                 'test_mock_data_file.csv')

set_sequence_in_database('test_db', 'test_table_id_seq', NUMBER_OF_TEST_RECORDS_TO_GENERATE + 1)

# Inspect the data inserted in to the database

In [ ]:
query_database_return_dataframe('test_db', EXAMPLE_QUERY)


# Truncate data in database table


In [ ]:
truncate_data_in_database('test_db',
                          'test_table')